In [ ]:
#Params
param_stationID = '270'
param_from_date = '2022'
param_to_date = '2023' 
param_vars = 'R'

In [10]:
install.packages("readr")
install.packages("dplyr")
install.packages("tidyr")
install.packages("stringr")
install.packages("lubridate")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [ ]:
#KNMI data retriever


#Parameter validation 
if (param_stationID == "" ||
    param_from_date == "" ||
    param_to_date == "" ||
    param_vars == "") {
  stop("All parameters must be provided by the user.")
}

stationID  <- param_stationID
from       <- param_from_date
to         <- param_to_date

#Function
get_daily_data <- function(stationID, from, to, vars) {

#Date parsing 
    if (!is.character(from) ||
      !is.character(to) ||
      stringr::str_length(from) %% 2 == 1 ||
      stringr::str_length(to) %% 2 == 1) {
    stop(
      "The values for 'from' and 'to' must be strings in the format 'YYYY', 'YYYYMM' or 'YYYYMMDD'."
    )
  }

  if (stringr::str_length(from) == 6) {
    from_date <- paste0(from, "01")
  } else if (stringr::str_length(from) == 4) {
    from_date <- paste0(from, "0101")
  } else {
    from_date <- from
  }

  if (stringr::str_length(to) == 8) {
    to_date <- to
  } else {
    if (stringr::str_length(to) == 6) {
      to <- paste0(to, "01")
    } else if (stringr::str_length(to) == 4) {
      to <- paste0(to, "1231")
    }

    to_date <- lubridate::ymd(to) %>%
      lubridate::ceiling_date(unit = "month") - 1

    to_date <- gsub("-", "", as.character(to_date))
  }

  if (as.numeric(to_date) < as.numeric(from_date)) {
    stop("'from' must be earlier than or equal to 'to'")
  }

  # Station validation
  selectable_stations <- c("260", "275")  # De Bilt, Deelen
  stationID <- as.character(stationID)

  if (!stationID %in% selectable_stations) {
    stop(
      paste0(
        "Invalid stationID. Choose one of: ",
        paste(selectable_stations, collapse = ", ")
      )
    )
  }

#Build KNMI URL
baseURL <- "https://www.daggegevens.knmi.nl/klimatologie/daggegevens"
  URL <- paste0(
    baseURL,
    "?start=", from_date,
    "&end=", to_date,
    "&stns=", stationID,
    "&ALL"
  )

  #Download data from KNMI 
  data_daily <- readr::read_csv(
    URL,
    col_names = FALSE,
    comment = "#",
    show_col_types = FALSE
  ) %>%
    dplyr::as_tibble()

  colnames(data_daily) <- c(
    "STN", "YYYYMMDD", "FG", "TG", "TN",
    "TX", "RH", "DDVEC", "UG", "UX", "UN"
  )

  # Selected requested variables
  data_daily <- data_daily %>%
    tidyr::drop_na(dplyr::all_of(vars)) %>%
    select(dplyr::all_of(c("YYYYMMDD", vars)))

  # Scaling because KNMI works in 0.1
  if ("FG" %in% vars) data_daily$FG <- data_daily$FG / 10
  if ("TG" %in% vars) data_daily$TG <- data_daily$TG / 10
  if ("TN" %in% vars) data_daily$TN <- data_daily$TN / 10
  if ("TX" %in% vars) data_daily$TX <- data_daily$TX / 10
  if ("RH" %in% vars) data_daily$RH <- data_daily$RH / 10

  return(data_daily)
}

#Run function for climate data 
KNMI_datafile <- ''
KNMI_datafile <- get_daily_data(
  stationID = stationID,
  from      = from,
  to        = to,
  vars      = vars
)

#Run function for spin up climate data 
KNMI_spinup_datafile <- ""
KNMI_spinup_datafile <- get_daily_data(
  stationID = stationID,
  from      = from,
  to        = to,
  vars      = vars
)

In [ ]:
# KNMI data preprocessor
Preprocessed_KNMI_datafile <- ""
Preprocessed_KNMI_datafile <- KNMI_datafile

Preprocessed_KNMI_spinup_datafile <- ""
Preprocessed_KNMI_spinup_datafile <- KNMI_spinup_datafile

In [ ]:
# Species trait data retriever
Species_trait_datafile <- "Species_trait_datafile"

In [ ]:
# Species trait data preprocessor
Preprocessed_species_trait_datafile <- ""
Preprocessed_species_trait_datafile <- Species_trait_datafile

In [ ]:
# Initial community data retriever
Initial_community_datafile <- "Initial community datafile"

In [ ]:
# Initial community data proprocessor
Preprocessed_initial_community_datafile <- ""
Preprocessed_initial_community_datafile <- Initial_community_datafile

In [ ]:
# Scenario and ecoregions builder
print(Preprocessed_KNMI_datafile)
print(Preprocessed_species_trait_datafile)
print(Preprocessed_initial_community_datafile)
print(Preprocessed_KNMI_spinup_datafile)
Scenario_file <- "scenario file"

In [ ]:
# LANDIS II runner
print(Scenario_file)